<a href="https://colab.research.google.com/github/wowrainbow/quest_c1/blob/main/0827q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [1]:
import tensorflow
import numpy as np
import transformers
import argparse

print(tensorflow.__version__)
print(np.__version__)
print(transformers.__version__)
print(argparse.__version__)

2.17.0
1.26.4
4.42.4
1.1


In [2]:
import tensorflow_datasets as tfds

dataset = tfds.load('glue/mnli')

In [3]:
dataset

{'train': <_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'validation_matched': <_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'validation_mismatched': <_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'test_matched': <_PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': Te

In [4]:
import pandas as pd

pd.DataFrame(dataset['train'].take(5))
#가져온 데이터셋의 형태를 확인했다.

,hypothesis,idx,label,premise
0,tf.Tensor(b'Meaningful partnerships with stake...,"tf.Tensor(16399, shape=(), dtype=int32)","tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b'In recognition of these tensions, ..."
1,tf.Tensor(b'The Clinton followers kept to the ...,"tf.Tensor(206287, shape=(), dtype=int32)","tf.Tensor(0, shape=(), dtype=int64)",tf.Tensor(b'The Clinton surrogates also held t...
2,tf.Tensor(b'Women have jobs in all areas of th...,"tf.Tensor(352707, shape=(), dtype=int32)","tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b""um-hum because women are in every ..."
3,tf.Tensor(b'Houston is freezing and dry right ...,"tf.Tensor(372070, shape=(), dtype=int32)","tf.Tensor(2, shape=(), dtype=int64)","tf.Tensor(b'Houston is really humid now', shap..."
4,"tf.Tensor(b""But they wouldn't be leaving right...","tf.Tensor(160184, shape=(), dtype=int32)","tf.Tensor(1, shape=(), dtype=int64)","tf.Tensor(b'But not now.', shape=(), dtype=str..."


# STEP 2. MNLIProcessor

In [5]:
class MNLIProcessor():
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.examples = []
        self.max_examples = 50000 # 35만개중 5만개까지만 선택했다.

    def get_example_from_tensor_dict(self, tensor_dict):
        return (
            tensor_dict['idx'].numpy(),
            tensor_dict['premise'].numpy().decode('utf-8'),
            tensor_dict['hypothesis'].numpy().decode('utf-8'),
            tensor_dict['label'].numpy()
        )

    def prepare_examples(self, tensor_dicts):
        self.examples = []
        for tensor_dict in tensor_dicts:
            if len(self.examples) >= self.max_examples:
                break
            example = self.get_example_from_tensor_dict(tensor_dict)
            self.examples.append(example)
        return self.examples

In [6]:
processor = MNLIProcessor()

for example in dataset['train'].take(1):
    example = processor.get_example_from_tensor_dict(example)

In [7]:
example

(16399,
 'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.',
 'Meaningful partnerships with stakeholders is crucial.',
 1)

In [8]:
example_list = processor.prepare_examples(dataset['train']) #리스트

# STEP 3. 토크나이저

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli', padding=True, truncation=True) #이름에 mnli가 들어가있어서 선택

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
tokenized_example = []
for example in example_list:
    text = example[1]
    tokenized_example.append(tokenizer(text))

In [11]:
tokenized_example[0]
#토크나이징이 잘 되었다.

{'input_ids': [0, 1121, 4972, 9, 209, 5734, 6, 226, 3632, 34, 1006, 28554, 187, 7969, 7, 15451, 5, 2113, 9, 5, 331, 9619, 11980, 8, 7, 5242, 6667, 8670, 19, 7193, 3448, 23, 26242, 10, 92, 39835, 38115, 227, 5, 752, 3696, 8, 11940, 9, 1030, 518, 1435, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
len(tokenized_example)

50000

# STEP 4. 모델 생성

In [13]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

model = TFAutoModelForSequenceClassification.from_pretrained('roberta-large-mnli')

All PyTorch model weights were used when initializing TFRobertaForSequenceClassification.

All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  354310144 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  1052675   
 ificationHead)                                                  
                                                                 
Total params: 355362819 (1.32 GB)
Trainable params: 355362819 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#잘안됨